<p style="background: orange;  color: blue; text-align: center;letter-spacing: 2px; line-height: 150%; font-weight:bold; font-size:xx-large;">The Partition Transform</p>
<p style="background: color: blue; text-align: center;">Peter Luschny, March 2016<br> updated September 2022</p>

<p>The notebook can be downloaded from
<a href = "https://github.com/PeterLuschny/PartitionTransform/blob/main/PartitionTransform.ipynb">GitHub</a>. The notebook requires a SageMath kernel and was tested with SageMath 9.6. If you do not have (the free and open) SageMath installed, we recommend using it in the cloud at <a href="https://cocalc.com/">CoCalc</a>.
The notebook was designed in connection with my blog at the 
<a href = "http://oeis.org/wiki/User:Peter_Luschny/P-Transform">OEIS-Wiki</a>.</p> 

In [1]:
%load_ext sage

import operator
from enum import Enum
from functools import cache
from typing import Callable
from itertools import accumulate
from inspect import getsource
from sage.combinat.partition import Partition, Partitions
from sage.all import ZZ, QQ, factorial, binomial, product, matrix
from sage.all import rising_factorial, falling_factorial, var
from sage.all import stirling_number1, stirling_number2, bernoulli_polynomial
from sage.all import SR, PolynomialRing, expand, flatten

This notebook compiles a small toolbox for studying the *partition transformation*. The partition transformation is a sequence-to-triangle transformation.
Here we understand by a triangle a lower infinite triangular array 
$T(n, k)$ for $0 \le k \le n$, where the $T(n,k)$ are integers or rational numbers.

We introduce four main functions:

* PartitionPolynomials
* EvalPartPoly
* PartitionTransform
* PartitionSum

The first function, *PartitionPolynomials*, defines the problem abstractly as multivariate polynomials and considers their coefficients, decomposition into partial polynomials and monomials. It also defines what we mean by the inverses of these polynomials.

The second function, *EvalPartPoly*, interprets the multivariate polynomials; that is, it evaluates them by substituting the variables with the values of a given sequence. This transforms the partial polynomials into the number triangles.

The third function, *PartitionTransform*, puts all this conceptual frame aside and calculates the number triangles directly and recursively from the given sequence. It is thus computationally much more efficient and the real workhorse in the numerical representation of the triangles.

The fourth function, *PartitionSum*, is ironically the first to use partitions explicitly. But it is closest to the historical core of the theory, has given it the name used here, and invites an experimental play with the formulas.

In the final section, we give various examples of the use of the transformation and provide cross-references to the OEIS. We also demonstrate the use of the transformation in a generalization of the Stirling-Lah family of number sequences and consider new relatives of the Ward numbers.

The notebook was written in 2016 as a companion to a blog post on the author's pages on the OEIS and revised in September 2022. In the meantime, Cormac O'Sullivan published an excellent exposition of this transformation and its history on the arXiv. O'Sullivan proposes to call the partition transformation the *De Moivres polynomials*. 

Cormac O'Sullivan, <a href="https://arxiv.org/abs/2203.02868">De Moivre and Bell polynomials</a>, arXiv:2203.02868 [math.CO], 2022. 

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
PART 1 Partition Polynomials</h3>

In this part we give all definitions in an abstract way. The enumaration below gives a fairly complete overview about the different aspect we will deal with.

In [2]:
#  Representations of the partition polynomials:
class PP_Repr(Enum):                                  # -- returned type
    COMPPOLY  = 1  # COMPLETE POLYNOMIAL                -- polynomial (multi variable)
    PARTPOLY  = 2  # PARTIAL POLYNOMIAL                 -- list of polynomials
    MONOLIST  = 3  # LIST OF MONOMIALS                  -- list of monomials
    PARTCOEFF = 4  # COEFFICIENT-LIST OF PARTPOLYS      -- list of list[int]
    COMPCOEFF = 5  # COEFFICIENT-LIST OF DICTPOLY       -- list[int], length = A000041(n)
    PARTSUM   = 6  # PARTIAL POLYNOMIALS EVALUATED AT 1 -- list[int], length = n
    COMPSUM   = 7  # COMPLETE POLYNOMIAL EVALUATED AT 1 -- int
    UNOPOLY   = 8  # POLYNOMIAL IN ONE VARABLE          -- list of single variable polys
    MATRIX    = 9  # MATRIX OF PARTIAL POLYNOMIALS      -- matrix
    INVMATRIX = 10 # INVERSE MATRIX OF PART POLYNOMIALS -- matrix
    GENSEQ    = 11 # GENERATING SEQUENCE                -- list[int]
    INVSEQ    = 12 # INVERSE OF GENERATING SEQUENCE     -- list[int]

<pre style="color:darkgreen; font-weight:600;">
PartitionPolynomials(
    dim: int, 
    rep: PP_Repr, 
    ord='lex'
) -> list:
</pre>

* The parameter *dim* gives the number of complete polynomials considered, respectively the dimension (number of rows) of the triangle of the partial polynomials.
* The parameter *rep* is an element of the enumeration given above and indicates the required representation.
* The third parameter is optional and indicates the ordering of the monomials. Throughout this notebook it will be fixed to the lexicographical order. But be aware that the ordering used in referenced triangles on the OEIS might be different.

In [3]:
def PartitionPolynomials(dim: int, rep: PP_Repr, ord='lex') -> list:

    X = var(['x' + str(i) for i in range(dim + 1)])
    
    @cache
    def PartPoly(n: int, k: int):
        Z = X[1: n - k + 2]
        R = PolynomialRing(ZZ, Z, n - k + 1, order=ord)
        if k == 0: return R(k^n)
        return  R(sum(PartPoly(n - j, k - 1) * Z[j - 1] 
                for j in range(1, n - k + 2)).expand())


    @cache
    def PolyRow(n: int): 
        return [PartPoly(n, k) for k in range(n + 1)]


    def DictRow(n: int):
        R = PolyRow(n)
        row = [[p.monomials(), p.coefficients()] for p in R]
        Z = [[a*b for (a, b) in zip(p[0], p[1])] for p in row]
        return flatten([t if t != [] else [0] for t in Z])


    def PartCoeffRow(n: int): 
        return [[0] if (c := p.coefficients()) == [] 
                    else c for p in PolyRow(n)]


    def CoeffRow(n: int): 
        return flatten([[0] if (c := p.coefficients()) == [] 
                    else c for p in PolyRow(n)])


    def ReducedRow(n: int): 
        return flatten([0 if (c := p.coefficients()) == [] 
                    else sum(c) for p in PolyRow(n)])


    def UnoPoly(coefs: list[int]):
        R = PolynomialRing(ZZ, 'x'); x = R.gen()
        return R(sum(c * x^k for (k, c) in enumerate(coefs)))


    @cache
    def PolyMatrix(dim: int): 
        return matrix([[PartPoly(n, k) if k <= n else 0 
                for k in range(dim)] for n in range(dim)])


    print(); print(rep.name)


    if rep == PP_Repr.COMPPOLY:
        return [sum(PolyRow(n)) for n in range(dim)]


    elif rep == PP_Repr.PARTPOLY:
        return [PolyRow(n) for n in range(dim)]


    elif rep == PP_Repr.MONOLIST:
        return [DictRow(n) for n in range(dim)]


    elif rep == PP_Repr.PARTCOEFF:
        return [PartCoeffRow(n) for n in range(dim)]


    elif rep == PP_Repr.COMPCOEFF:
        return [CoeffRow(n) for n in range(dim)]


    elif rep == PP_Repr.PARTSUM:
        return [ReducedRow(n) for n in range(dim)]


    elif rep == PP_Repr.COMPSUM:
        return [sum(ReducedRow(n)) for n in range(dim)]


    elif rep == PP_Repr.UNOPOLY:
        return [UnoPoly(ReducedRow(n)) for n in range(dim)]


    elif rep == PP_Repr.MATRIX:
        return PolyMatrix(dim)


    elif rep == PP_Repr.INVMATRIX:
        return PolyMatrix(dim).inverse()


    elif rep == PP_Repr.GENSEQ:
        return X[1:dim]


    elif rep == PP_Repr.INVSEQ:
        M = PolyMatrix(dim).inverse()
        return [M[n + 1, 1] for n in range(dim - 1)]


    return [[]]

An utility function which gives an overview.

In [4]:
def ShowAll(dim: int):
    for repr in PP_Repr:
        T = PartitionPolynomials(dim, rep=repr)
        for row in T: print(row)

ShowAll(7)


COMPPOLY
1
x1
x1^2 + x2
x1^3 + 2*x1*x2 + x3
x1^4 + 3*x1^2*x2 + 2*x1*x3 + x2^2 + x4
x1^5 + 4*x1^3*x2 + 3*x1^2*x3 + 3*x1*x2^2 + 2*x1*x4 + 2*x2*x3 + x5
x1^6 + 5*x1^4*x2 + 4*x1^3*x3 + 6*x1^2*x2^2 + 3*x1^2*x4 + 6*x1*x2*x3 + 2*x1*x5 + x2^3 + 2*x2*x4 + x3^2 + x6

PARTPOLY
[1]
[0, x1]
[0, x2, x1^2]
[0, x3, 2*x1*x2, x1^3]
[0, x4, 2*x1*x3 + x2^2, 3*x1^2*x2, x1^4]
[0, x5, 2*x1*x4 + 2*x2*x3, 3*x1^2*x3 + 3*x1*x2^2, 4*x1^3*x2, x1^5]
[0, x6, 2*x1*x5 + 2*x2*x4 + x3^2, 3*x1^2*x4 + 6*x1*x2*x3 + x2^3, 4*x1^3*x3 + 6*x1^2*x2^2, 5*x1^4*x2, x1^6]

MONOLIST
[1]
[0, x1]
[0, x2, x1^2]
[0, x3, 2*x1*x2, x1^3]
[0, x4, 2*x1*x3, x2^2, 3*x1^2*x2, x1^4]
[0, x5, 2*x1*x4, 2*x2*x3, 3*x1^2*x3, 3*x1*x2^2, 4*x1^3*x2, x1^5]
[0, x6, 2*x1*x5, 2*x2*x4, x3^2, 3*x1^2*x4, 6*x1*x2*x3, x2^3, 4*x1^3*x3, 6*x1^2*x2^2, 5*x1^4*x2, x1^6]

PARTCOEFF
[[1]]
[[0], [1]]
[[0], [1], [1]]
[[0], [1], [2], [1]]
[[0], [1], [2, 1], [3], [1]]
[[0], [1], [2, 2], [3, 3], [4], [1]]
[[0], [1], [2, 2, 1], [3, 6, 1], [4, 6], [5], [1]]

COMPCOEFF
[1]
[0, 1]

In [5]:
T = PartitionPolynomials(7, rep=PP_Repr.MATRIX)
S = PartitionPolynomials(7, rep=PP_Repr.INVMATRIX)
print(T * S)


MATRIX

INVMATRIX
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]


The matrix of the partial partition polynomials:


$$  \begin{array}{ccccccc}
     0 & 1 & 2 & 3 & 4 & 5 & 6 \\ 
    \hline \\
    1 & . & . & . & . & . & . \\ 
    0 & x_1 & . & . & . & . & . \\ 
    0 & x_2 & x_1^2 & . & . & . & . \\ 
    0 & x_3 & 2x_1x_2 & x_1^3 & . & . & . \\ 
    0 & x_4 & 2x_1x_3 + x_2^2 & 3x_1^2x_2 & x_1^4 & . & . \\ 
    0 & x_5 & 2x_1x_4 + 2x_2x_3 & 3x_1^2x_3 + 3x_1x_2^2 & 4x_1^3x_2 & x_1^5 & . \\ 
    0 & x_6 & x_3^2 + 2x_2x_4 + 2x_1x_5 & x_2^3 + 6x_1x_2x_3 + 3x_1^2x_4  & 6x_1^2x_2^2 + 4x_1^3x_3 &  5x_1^4x_2 & x_1^6  
\end{array}  $$

Assuming $x_1 = 1$:

$$ \begin{array}{ccccc}
    0 & 1 & 2 & 3 & 4 & 5 & 6 \\ 
    \hline \\
    1 & . & . & . & . & . & . \\ 
    0 & 1 & . & . & . & . & . \\ 
    0 & x_2 & 1 & . & . & . & . \\ 
    0 & x_3 & 2x_2 & 1 & . & . & . \\ 
    0 & x_4 & 2x_3 + x_2^2 & 3x_2 & 1 & . & . \\ 
    0 & x_5 & 2x_4 + 2x_2x_3 & 3x_3 + 3x_2^2 & 4x_2 & 1 & . \\ 
    0 & x_6 & x_3^2 + 2x_2x_4 + 2x_5 & x_2^3 + 6x_2x_3 + 3x_4 & 6x_2^2 + 4x_3 & 5x_2 & 1 \\ 
\end{array} $$

Polynomial rings have monomial orderings. We use the lexigrophical by default. Sage uses the degrevlex order by default.

The terms of the polynomials are automatically ordered from greatest to smallest (with respect to the monomial ordering).

<center><table style="text-align:center">
    <tr style="font-weight:650">
        <td>n\k</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>3</td>
        <td>4</td>
        <td>5</td>
        <td>6</td>
    </tr>
    <tr>
        <td><b>0</b></td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>1</b></td>
        <td>0</td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>2</b></td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>3</b></td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>4</b></td>
        <td>0</td>
        <td>1</td>
        <td>2, 1</td>
        <td>3</td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>5</b></td>
        <td>0</td>
        <td>1</td>
        <td>2, 2</td>
        <td>3, 3</td>
        <td>4</td>
        <td>1</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>6</b></td>
        <td>0</td>
        <td>1</td>
        <td>1, 2, 2</td>
        <td>1, 6, 3</td>
        <td>6, 4</td>
        <td>5</td>
        <td>1</td>
    </tr>
</table></center>

<p><a href="https://oeis.org/A269941">A269941</a> the coefficients of the partial P-polynomials in degrevlex order.</p>

We denote the inverse matrix of the partial partition polynomials $ \mathcal{P^{-1}} $. 
We also call $\mathcal{P^{-1}}(n,k)$ the inverse polynomial of the partial partition polynomial 
$ \mathcal{P}(n,k) $. 

$$ \begin{array}{ccccc}
  0 & 1 & 2 & 3 & 4 & 5  \\ 
    \hline \\
1 & . & . & . & . & . \\
0 & -\frac{1}{z_{1}} & . & . & . & . \\
0 & -\frac{z_{2}}{z_{1}^{3}} & \frac{1}{z_{1}^{2}} & . & . & . \\
0 & -\frac{2 \, z_{2}^{2} - z_{1} z_{3}}{z_{1}^{5}} 
& \frac{2 \, z_{2}}{z_{1}^{4}} & -\frac{1}{z_{1}^{3}} & . & . \\
0 & -\frac{5 \, z_{2}^{3} - 5 \, z_{1} z_{2} z_{3} + z_{1}^{2} z_{4}}{z_{1}^{7}} 
& \frac{5 \, z_{2}^{2} - 2 \, z_{1} z_{3}}{z_{1}^{6}} & -\frac{3 \, z_{2}}{z_{1}^{5}} & \frac{1}{z_{1}^{4}} & . \\
0 & -\frac{14 \, z_{2}^{4} - 21 \, z_{1} z_{2}^{2} z_{3} + 3 \, z_{1}^{2} z_{3}^{2} + 6 \, z_{1}^{2} z_{2} z_{4} - z_{1}^{3} z_{5}}{z_{1}^{9}} 
& \frac{{14 \, z_{2}^{3} - 12 \, z_{1} z_{2} z_{3} + 2 z_{1}^{2} z_{4} }}{z_{1}^{8}} & -\frac{{9 \, z_{2}^{2} - 3 z_{1} z_{3}}}{z_{1}^{7}} & \frac{4 \, z_{2}}{z_{1}^{6}} & -\frac{1}{z_{1}^{5}}
\end{array} $$

Assuming $z_1 = 1$:

$$ \begin{array}{ccccc}
  0 & 1 & 2 & 3 & 4 & 5 \\ 
    \hline \\
1 & . & . & . & . & . \\
0 & -{1} & . & . & . & . \\
0 & -{z_{2}} & {1} & . & . & . \\
0 & -({2 \, z_{2}^{2} - z_{3}}) & {2 \, z_{2}} & -{1} & . & . \\
0 & -({5 \, z_{2}^{3} - 5 \, z_{2} z_{3} + z_{4}}) & {5 \, z_{2}^{2} - 2 \, z_{3}} & -{3 \, z_{2}} & {1} & . \\
0 & -({14 \, z_{2}^{4} - 21 \, z_{2}^{2} z_{3} + 3 \, z_{3}^{2} + 6 \, z_{2} z_{4} - z_{5}}) & {{14 \, z_{2}^{3} - 12 \, z_{2} z_{3} + 2 z_{4} }} & -({{9 \, z_{2}^{2} - 3 z_{3}}}) & {4 \, z_{2}} & -{1}
\end{array} $$

<center><table style="text-align:center">
    <tr style="font-weight:650">
        <td>n\k</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>3</td>
        <td>4</td>
        <td>5</td>
    </tr>
    <tr>
        <td><b>0</b></td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>1</b></td>
        <td>0</td>
        <td>-1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>2</b></td>
        <td>0</td>
        <td>-1</td>
        <td>1</td>
        <td>-</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>3</b></td>
        <td>0</td>
        <td>-2, 1</td>
        <td>2</td>
        <td>-1</td>
        <td>-</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>4</b></td>
        <td>0</td>
        <td>-5, 5, -1</td>
        <td>5, -2</td>
        <td>-3</td>
        <td>1</td>
        <td>-</td>
    </tr>
    <tr>
        <td><b>5</b></td>
        <td>0</td>
        <td>-14, 21, -3, -6, 1</td>
        <td>14, -12, 2</td>
        <td>-9, 3</td>
        <td>4</td>
        <td>-1</td>
    </tr>
</table></center>

<p><a href="https://oeis.org/A269942">A269942</a> the coefficients of the inverse partial P-polynomials.</p>

The sequence P-inverse of the sequence $a = a_1, a_2, a_3,... $ are the terms which appear in column 1 of the inverse partition matrix of $a$. We will write this in short form:

$$ \mathcal{p^{-1}}(a) = [x^1] \mathcal{P^{-1}}(a) $$ 

Or, if full detail, the first few terms of the P-inverse sequence of $a$ are:
 
$$ \mathcal{p^{-1}}(a) \, = \,
-\frac{1}{a_1}, 
-\frac{a_{2}}{a_{1}^{3}} , 
-\frac{2 \, a_{2}^{2} - a_{1} a_{3}}{a_{1}^{5}},
-\frac{5 \, a_{2}^{3} - 5 \, a_{1} a_{2} a_{3} + a_{1}^{2} a_{4}}{a_{1}^{7}}, ...$$

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Evaluating the Partition Polynomials</h3>

Evaluating the partition polynomials by substituting

<pre style="color:darkgreen; font-weight:600;">
EvalPartPoly(
    dim: int, 
    f:   Callable, 
    rep: PP_Repr=PP_Repr.PARTPOLY
):
</pre>

* The parameter *dim* is the length of the generated sequence or the dimension (number of rows) of the generated triangel returned.
* The parameter *f* is the generating sequence, given in the form of a function.
* The optional parameter *rep* describes the desired representation and is set to PARTPOLY by default.

In [6]:
def EvalPartPoly(dim: int, f: Callable, rep: PP_Repr=PP_Repr.PARTPOLY):

    X = var(['x' + str(i) for i in range(dim + 1)])
    Y = [f(n) for n in range(dim +1)]

    def substitut(poly):
        if poly in ZZ: return poly
        S = {}
        for v in poly.variables():
            try:
                S[v] = Y[X.index(v)]
            except ValueError:
                continue
        return poly.substitute(S)

    P = PartitionPolynomials(dim, rep)

    if rep == PP_Repr.COMPPOLY or rep == PP_Repr.GENSEQ or rep == PP_Repr.INVSEQ:
        return [substitut(SR(p)) for p in P]

    elif rep == PP_Repr.MATRIX or rep == PP_Repr.INVMATRIX:
        return matrix([[substitut(SR(t)) for t in row] for row in P])

    return [[substitut(pp) for pp in p] for p in P]

The three examples that should be remembered are $f(n) = 1$, $f(n) = n$, and $f(n) = \operatorname{Catalan}(n)$:

<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A097805|id:A071919">A097805, A071919</a>: 
Number of compositions of n with k parts, and <br>
monotone nondecreasing functions [n] -> [k].
<span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A033999">A033999</a>, Grandi's series. </p>

In [7]:
def f(n): return 1
print(EvalPartPoly(9, f, rep=PP_Repr.INVSEQ))
EvalPartPoly(7, f)


INVSEQ
[1, -1, 1, -1, 1, -1, 1, -1]

PARTPOLY


[[1],
 [0, 1],
 [0, 1, 1],
 [0, 1, 2, 1],
 [0, 1, 3, 3, 1],
 [0, 1, 4, 6, 4, 1],
 [0, 1, 5, 10, 10, 5, 1]]

Because of the special importance of this case $(f \equiv 1)$, we give another representation which we will meet below in the context of the partition sum. Here $p \in P_{n, k}$ if $p$ is a partition of $n$ and the greatest part of $p$ is $k.$

In [8]:
def U(n, k):
    Pnk = Partitions(n, max_part=k, inner=[k])
    return sum([product(binomial(p[j], p[j+1]) 
                for j in range(len(p) - 1)) for p in Pnk])

for n in range(7): 
    print([U(n, k) for k in range(n + 1)])

[1]
[0, 1]
[0, 1, 1]
[0, 1, 2, 1]
[0, 1, 3, 3, 1]
[0, 1, 4, 6, 4, 1]
[0, 1, 5, 10, 10, 5, 1]


<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A128908|id:A285072|id:A193401">A128908, A285072, A193401</a>: Coefficients of the characteristic polynomials for the Cartan matrix of the root system A_n, or coefficients of the Laplacian polynomial of the n-path graph P_n.
<span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A000108">A000108</a>, signed Catalan numbers. </p>

In [9]:
def f(n): return n
print(EvalPartPoly(9, f, rep=PP_Repr.INVSEQ))
EvalPartPoly(7, f)


INVSEQ
[1, -2, 5, -14, 42, -132, 429, -1430]

PARTPOLY


[[1],
 [0, 1],
 [0, 2, 1],
 [0, 3, 4, 1],
 [0, 4, 10, 6, 1],
 [0, 5, 20, 21, 8, 1],
 [0, 6, 35, 56, 36, 10, 1]]

<p>For the next example search the OEIS with the strings "132, 165, 110, 44, 10, 1" and "1, 10, 44, 110, 165, 132" and you will get more than twenty hits.</p>
<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A128899|id:A039598|id:A050166">A128899, A039598, A050166</a>: <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A181983">A181983</a>, a(n) = (-1)^(n+1) * n. </p>

In [10]:
def f(n): return binomial(2*n, n) // (n + 1)
print(EvalPartPoly(9, f, rep=PP_Repr.INVSEQ))
EvalPartPoly(7, f)


INVSEQ
[1, -2, 3, -4, 5, -6, 7, -8]

PARTPOLY


[[1],
 [0, 1],
 [0, 2, 1],
 [0, 5, 4, 1],
 [0, 14, 14, 6, 1],
 [0, 42, 48, 27, 8, 1],
 [0, 132, 165, 110, 44, 10, 1]]

One gets a full numerical overview with this convenience function:

In [11]:
def EvalAll(dim: int, f: Callable):
    for rep in [PP_Repr.PARTPOLY, PP_Repr.MONOLIST, 
                PP_Repr.MATRIX, PP_Repr.INVMATRIX, 
                PP_Repr.COMPPOLY, PP_Repr.GENSEQ, PP_Repr.INVSEQ]:
        print(EvalPartPoly(dim, f, rep))

TEST

In [12]:
TestFunctions = [
    lambda n: 1,
    lambda n: -1,
    lambda n: (-1)^n,
    lambda n: n,
    lambda n: -n,
    lambda n: (-1)^n * n,
    lambda n: factorial(n),
    lambda n: -factorial(n),
    lambda n: (-1)^n * factorial(n),
    lambda n: binomial(2*n, n) // (n + 1),
    lambda n: -binomial(2*n, n) // (n + 1),
    lambda n: (-1)^n * binomial(2*n, n) // (n + 1)
]

In [13]:
def PolyTest(dim):
    for f in TestFunctions: 
        print("\n" + "=" * 70)
        print(getsource(f))
        EvalAll(dim, f)

# PolyTest(7)  # to execute the test remove comment sign

This closes part 1, where we considered the essential notions related to the partition polynomials and a general method of evaluating these polynomials.

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
PART 2 Partition Transform</h3>

In this section, we change our view of the polynomials and consider the variables of the polynomials as the terms of a given sequence. In other words, we look at the partial polynomials as generators of a sequence transformation. We will focus on efficient evaluation and add some extensions.

<pre style="color:darkgreen; font-weight:600;">
def PartitionTransform(
    dim:     int, 
    a:       Callable=None, 
    norm:    Callable=None, 
    inverse: bool=False, 
    accu:    bool=False
) -> list:
</pre>

* The only mandatory parameter is $\operatorname{dim}$, the number of rows of the triangle.  

* $a = a_1, a_2, a_3, ...$ is the generating sequence, it has offset 1. The sequence is given as a function $a(n) = n$. If $a$ is not given, the symbols $x_1, x_2, x_3, ...$ are used. In contrast to the situation in the last section, however, these are not elements of the PolynomialRing. For instance, in the function PartitionPolynomials $x_2^2 + 2x_1x_3$ was an element of the PolynomialRing $(Z, [x_1, x_2, x_3])$, here it is only a symbolic expression.

* The third and optional parameter $\operatorname{norm}$ is a function in $n$ and $k$, with which the terms $T(n, k)$ are normalized (multiplicative) after the transformation. If the generating sequence is rational, the target matrix is usually rational as well. With the $\operatorname{norm}$ function it can then often be recasted to an integer triangle.

* The inverse triangle is generated if the optional parameter $\operatorname{inverse}$ is set to True. 

* If the optional parameter $\operatorname{accu}$ is set to True, the multiplicative accumulated $a$ is used instead of $a$.

<table style="width: 100%; teat-align:center; background-color:#CDE1BA "> 
<caption style="font-size:larger;">The transformation in its accumulated form.</caption>
<tr style="background-color:#FFD700; color:blue">
<td>1</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr style="background-color:#FFD700; color:blue">
<td>0</td>
<td>a<sub>1</sub></td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr style="background-color:#FFD700; color:blue">
<td>0</td>
<td>a<sub>1</sub>a<sub>2</sub></td>
<td>a<sub>1</sub><sup>2</sup></td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr style="background-color:#FFD700; color:blue">
<td>0</td>
<td>a<sub>1</sub>a<sub>2</sub>a<sub>3</sub></td>
<td>2a<sub>1</sub><sup>2</sup>a<sub>2</sub></td>
<td>a<sub>1</sub><sup>3</sup></td>
<td>0</td>
<td>0</td>
</tr>
<tr style="background-color:#FFD700; color:blue">
<td>0</td>
<td>a<sub>1</sub>a<sub>2</sub>a<sub>3</sub>a<sub>4</sub></td>
<td>2a<sub>1</sub><sup>2</sup>a<sub>2</sub>a<sub>3</sub>+a<sub>1</sub><sup>2</sup>a<sub>2</sub><sup>2</sup></td>
<td>3a<sub>1</sub><sup>3</sup>a<sub>2</sub></td>
<td>a<sub>1</sub><sup>4</sup></td>
<td>0</td>
</tr>
<tr style="background-color:#FFD700; color:blue">
<td>0</td>
<td>a<sub>1</sub>a<sub>2</sub>a<sub>3</sub>a<sub>4</sub>a<sub>5</sub></td>
<td>2a<sub>1</sub><sup>2</sup>a<sub>2</sub>a<sub>3</sub>a<sub>4</sub>+2a<sub>1</sub> <sup>2</sup>a<sub>2</sub><sup>2</sup>a<sub>3</sub></td>
<td>3a<sub>1</sub><sup>3</sup>a<sub>2</sub>a<sub>3</sub>
+3a<sub>1</sub><sup>3</sup>a<sub>2</sub><sup>2</sup></td>
<td>4a<sub>1</sub><sup>4</sup>a<sub>2</sub></td>
<td>a<sub>1</sub><sup>5</sup></td>
</tr>
</table>


Besides the additional possibilities given by the parameters $\operatorname{norm}$ and $\operatorname{accu}$, the main advantage of this function compared to the function PartitionPolynomials is that it is much faster, which results from the fact that no typed levels are traversed. However, this is only noticeable if the dimension of the triangle is large. 

In [14]:
def PartitionTransform(dim: int, a: Callable=None, norm: Callable=None, 
        inverse: bool=False, accu: bool=False) -> list:

    if a == None: a = lambda n: var('x' + str(n))

    # Compensate the different indexing with a shift in the argument
    # and cache the input sequence.
    A = [a(i + 1) for i in range(dim - 1)]
    if accu: A = list(accumulate(A, operator.mul)) 
    print("In:", A)

    C = [[0 for k in range(m + 1)] for m in range(dim)]
    C[0][0] = 1

    if inverse:
        for m in range(1, dim):
            C[m][m] = C[m - 1][m - 1] / A[0]
            for k in range(m - 1, 0, -1):
                C[m][k] = expand(C[m - 1][k - 1] - 
                          sum(A[i] * C[m][k + i] 
                              for i in range(m - k + 1))) / A[0]
    else:
        for m in range(1, dim):
            C[m][m] = C[m - 1][m - 1] * A[0]
            for k in range(m - 1, 0, -1):
                C[m][k] = expand(sum(A[i] * C[m - i - 1][k - 1] 
                                    for i in range(m - k + 1)))

    if norm == None: return C
    for m in range(1, dim):
        for k in range(1, m + 1):
            C[m][k] = C[m][k] * norm(m, k)
    return C


def CompositionalInverse(dim, a):
    M = PartitionTransform(dim + 1, a, inverse=True)
    return [M[n + 1][1] for n in range(dim)]

In [15]:
def TransTest(dim):
    for f in TestFunctions: 
        print("\n" + "=" * 70)
        print(getsource(f))
        for row in PartitionTransform(dim, f): print(row)
        print("\nINVTRI")
        for row in PartitionTransform(dim, f, inverse=True): print(row)
        print("\nACCUTRI")
        for row in PartitionTransform(dim, f, accu=True): print(row)
        print("\nINVACCUTRI")
        for row in PartitionTransform(dim, f, accu=True, inverse=True): print(row)
        print("\nINVSEQ")
        print(CompositionalInverse(dim, f))

# TransTest(7) # to execute the test remove comment sign

In [16]:
PartitionTransform(5)
# PartitionTransform(5, accu=True)
# PartitionTransform(5, inverse=True)
# PartitionTransform(5, accu=True, inverse=True)

In: [x1, x2, x3, x4]


[[1],
 [0, x1],
 [0, x2, x1^2],
 [0, x3, 2*x1*x2, x1^3],
 [0, x4, x2^2 + 2*x1*x3, 3*x1^2*x2, x1^4]]

<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="http://oeis.org/A128908">A128908</a>, the Riordan array (1, x/(1-x)^2).</p>

In [17]:
PartitionTransform(7, lambda n: -n)

In: [-1, -2, -3, -4, -5, -6]


[[1],
 [0, -1],
 [0, -2, 1],
 [0, -3, 4, -1],
 [0, -4, 10, -6, 1],
 [0, -5, 20, -21, 8, -1],
 [0, -6, 35, -56, 36, -10, 1]]

<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="http://oeis.org/A090238">A090238</a>, 
Number of lists of k unlabeled permutations whose total length is n.
</p>

In [18]:
PartitionTransform(7, lambda n: n, accu=True)

In: [1, 2, 6, 24, 120, 720]


[[1],
 [0, 1],
 [0, 2, 1],
 [0, 6, 4, 1],
 [0, 24, 16, 6, 1],
 [0, 120, 72, 30, 8, 1],
 [0, 720, 372, 152, 48, 10, 1]]

<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A128899|id:A039598|id:A008313">A128899, A039598, A008313</a>, 
Riordan array (1, (1 - 2x - sqrt(1 - 4x))/(2x)), <br>Expansions of powers of x in terms of Chebyshev polynomials U_n(x). </p>

In [19]:
PartitionTransform(7, lambda n: n, lambda n,k: (-1)^(n - k), inverse=True)

In: [1, 2, 3, 4, 5, 6]


[[1],
 [0, 1],
 [0, 2, 1],
 [0, 5, 4, 1],
 [0, 14, 14, 6, 1],
 [0, 42, 48, 27, 8, 1],
 [0, 132, 165, 110, 44, 10, 1]]

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A000108">A000108</a>, signed Catalan numbers. </p> 

In [20]:
CompositionalInverse(9, lambda n: n)
# CompositionalInverse(9, lambda n: -n)

In: [1, 2, 3, 4, 5, 6, 7, 8, 9]


[1, -2, 5, -14, 42, -132, 429, -1430, 4862]

In [21]:
CompositionalInverse(9, lambda n: binomial(2 * n, n) // (n+1))

In: [1, 2, 5, 14, 42, 132, 429, 1430, 4862]


[1, -2, 3, -4, 5, -6, 7, -8, 9]

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A059372">A059372</a>, so-called "revert transform" of factorials n! (n >= 1).</p>

In [22]:
CompositionalInverse(9, lambda n: -factorial(n))

In: [-1, -2, -6, -24, -120, -720, -5040, -40320, -362880]


[-1, -2, -2, -4, 4, -48, 336, -2928, 28144]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
PART 3 The Partition Sum<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/P-Transform">OEIS-blog</a>]</span></h3>

We introduce a **sequence-to-triangle transformation** which we will call 
the *partition sum* (or short P-sum).
The name derives from the sum representation, which runs over all partitions of n.

We say, $p$ is an integer partition of $n$ or 
$p \in P_{n}$ 
$\Longleftrightarrow$ 
* $p = (p_{1}, \ldots, p_{n}), \ p_{i}$ integer, 
* $\sum_{1\le i \le n} p_{i}  = n$ and 
* $p_{1} \ge p_{2}  \ge \ldots \ge p_{n} \ge 0.$ 

$p \in P_{n, k}$ $\Longleftrightarrow$  $p \in P_{n}$ and the greatest part of $p$ is $k$, i.e. $p_1 = k$.

Given a sequence $a = a_1, a_2, a_3, \ldots$ let

$$ \mathcal{P}_{n}(a) = \sum_{p \in P_n}  
\binom{p_1}{p_2}\binom{p_2}{p_3} \ldots \binom{p_{n-1}}{p_{n}} 
x^{p_1} a_1^{p_1} a_2^{p_2} \ldots a_n^{p_n}. $$

Note that both the enumeration of the terms of a partition and the given sequence start at index $1$. The partition can be extended by as much zeors after the last nonzero term to the right as might be convenient.

The function PartitionSum implements this formula.

<pre style="color:darkgreen; font-weight:600;">
def PartitionSum(
    n:    int, 
    a:    Callable, 
    norm: Callable=None, 
    v:    int=1 | x
) -> list:
</pre>

* $n \ge 0$ is the integer over whose partitions is summed.
* $ a = a_1, a_2, a_3, ...$ is the generating sequence, given as a function $a(n)$.
* The third and optional parameter $\operatorname{norm}$ is a function in $n$ and $k$, with which the terms $P_{n,k}(a)$ are multiplied after the transformation. 
* The optional parameter $v$ has the default value $1$ but can assume any other value or take the symbolic value $v=x$.

The function returns the triangle $P_{n,k}(a)$, $0 \le k \le n$, evaluated at $v$ if $v$ is an integer or a rational value; it returns a symbolic expression representing a polynomial in $x$ if it is called with $v=x$.

Comparing the functions PartitionSum and PartitionTransform, note that 
in order for the function PartitionTransform to return the same result, the option "accu=True" must be set. In other words, the function PartitionSum has the accumulating effect build in.

In [23]:
def PartitionSum(n: int, a: Callable, norm: Callable=None, v: int=1) -> list:
    if n == 0: return 1 if v == x else [1]
    R = [0] * (n + 1)

    # we have to shift the index of the sequence by one place
    # because partitions are indexed in Sage starting with 0 
    for q in Partitions(n):
        p = q + [0]
        R[q[0]] += (v^q[0] * 
            product(binomial(p[j], p[j + 1]) * a(j + 1)^p[j] 
                    for j in range(len(q))))
    
    if norm != None: 
        R = [norm(n, k) * R[k] for k in range(n + 1)]
    return sum(R) if v == x else R


def PSum(n: int, f: Callable, v, norm: Callable=None) -> list:
    return v^(-n) * sum(PartitionSum(n, f, norm, v=v))

We have already met the case $(a \equiv 1)$ above. <p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span><a href="https://oeis.org/search?q=id:A097805|id:A071919"> A097805, A071919</a>: Number of compositions of n with k parts, and 
monotone nondecreasing functions [n] -> [k]. </p>

In [24]:
for n in range(7):
    print(PartitionSum(n, lambda n: 1))

[1]
[0, 1]
[0, 1, 1]
[0, 1, 2, 1]
[0, 1, 3, 3, 1]
[0, 1, 4, 6, 4, 1]
[0, 1, 5, 10, 10, 5, 1]


<p> <span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="http://oeis.org/A090238">A090238</a>, 
Number of lists of k unlabeled permutations whose total length is n.
</p>

In [25]:
f = lambda n: n

for n in range(7):
    print(PartitionSum(n, f))

[1]
[0, 1]
[0, 2, 1]
[0, 6, 4, 1]
[0, 24, 16, 6, 1]
[0, 120, 72, 30, 8, 1]
[0, 720, 372, 152, 48, 10, 1]


In [26]:
PartitionTransform(7, f, accu=True)

In: [1, 2, 6, 24, 120, 720]


[[1],
 [0, 1],
 [0, 2, 1],
 [0, 6, 4, 1],
 [0, 24, 16, 6, 1],
 [0, 120, 72, 30, 8, 1],
 [0, 720, 372, 152, 48, 10, 1]]

In [27]:
for n in range(7): print(PartitionSum(n, f, v=x))

1
x
x^2 + 2*x
x^3 + 4*x^2 + 6*x
x^4 + 6*x^3 + 16*x^2 + 24*x
x^5 + 8*x^4 + 30*x^3 + 72*x^2 + 120*x
x^6 + 10*x^5 + 48*x^4 + 152*x^3 + 372*x^2 + 720*x


In [28]:
for n in range(7): print(PartitionSum(n, f, v=x).list())

[1]
[0, 1]
[0, 2, 1]
[0, 6, 4, 1]
[0, 24, 16, 6, 1]
[0, 120, 72, 30, 8, 1]
[0, 720, 372, 152, 48, 10, 1]


<span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A051296">A051296</a>, so-called "invert transform" of factorials n!.</p>

In [29]:
print([PSum(n, f, 1) for n in range(10)])

[1, 1, 3, 11, 47, 231, 1303, 8431, 62391, 524495]


<span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A167894|id:A003319|id:A158882">A167894, A003319, A158882</a>, essentially the number of irreducible permutations.</p>

In [30]:
print([PSum(n, f, -1) for n in range(10)])

[1, 1, -1, 3, -13, 71, -461, 3447, -29093, 273343]


In [31]:
print([PSum(n, f, 1/2) for n in range(10)])

[1, 1, 5, 33, 269, 2633, 30421, 408945, 6307549, 110126809]


In [32]:
print([PSum(n, f, -1/2) for n in range(10)])

[1, 1, -3, 17, -139, 1449, -18131, 263233, -4339419, 80004377]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
PART 4 Partition representations of various sequences.</h3>

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>1</span></h2>

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A268434">A268434</a>, Lah numbers of order 2.</p>

In [33]:
lah2 = lambda n: 1 if n == 1 else ((n - 1)^2 + 1) / (n * (4 * n - 2))
norm = lambda n, k: factorial(2 * n) // factorial(2 * k)

for n in range(8): print(PartitionSum(n, lah2, norm))

[1]
[0, 1]
[0, 2, 1]
[0, 10, 10, 1]
[0, 100, 140, 28, 1]
[0, 1700, 2900, 840, 60, 1]
[0, 44200, 85800, 31460, 3300, 110, 1]
[0, 1635400, 3476200, 1501500, 203060, 10010, 182, 1]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>2</span></h2>
<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A241171">A241171</a>, Joffe's central differences of zero.</p>

In [34]:
eul = lambda n: 1 / ((2 * n - 1) * (2 * n))
nrm = lambda n, k: factorial(2 * n)

for n in range(8): print(PartitionSum(n, eul, nrm))

[1]
[0, 1]
[0, 1, 6]
[0, 1, 30, 90]
[0, 1, 126, 1260, 2520]
[0, 1, 510, 13230, 75600, 113400]
[0, 1, 2046, 126720, 1580040, 6237000, 7484400]
[0, 1, 8190, 1171170, 28828800, 227026800, 681080400, 681080400]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>3</span></h2>
<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href=""https://oeis.org/search?q=id:A269939|id:A134991">A269939, A134991</a>, Triangle of Ward numbers.</p>

In [35]:
ward = lambda n: 1 / (n + 1)
norm = lambda n, k: falling_factorial(n + k, n)

for n in range(8): print(PartitionSum(n, ward, norm))

[1]
[0, 1]
[0, 1, 3]
[0, 1, 10, 15]
[0, 1, 25, 105, 105]
[0, 1, 56, 490, 1260, 945]
[0, 1, 119, 1918, 9450, 17325, 10395]
[0, 1, 246, 6825, 56980, 190575, 270270, 135135]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Representations of the Euler and Bernoulli numbers<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/P-Transform#.E2.99.A6.C2.A0Representations_of_the_Euler_and_Bernoulli_numbers">OEIS-blog</a>]</span></h3>

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>4</span></h2>

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A046976|id:A046977">A046976/A046977</a>, Taylor series for sec(x) = 1/cos(x).</p>

In [36]:
eul = lambda n: 1 / ((2 * n - 1) * (2 * n))

print([PartitionSum(n, eul, v=x) for n in range(5)])
print([PartitionSum(n, eul, v=1) for n in range(5)])
print([sum(s) for s in [PartitionSum(n, eul, v=-1) for n in range(7)]])

[1, 1/2*x, 1/4*x^2 + 1/24*x, 1/8*x^3 + 1/24*x^2 + 1/720*x, 1/16*x^4 + 1/32*x^3 + 1/320*x^2 + 1/40320*x]
[[1], [0, 1/2], [0, 1/24, 1/4], [0, 1/720, 1/24, 1/8], [0, 1/40320, 1/320, 1/32, 1/16]]
[1, -1/2, 5/24, -61/720, 277/8064, -50521/3628800, 540553/95800320]


$$ E_{2n} = (2n)! \sum_{p \in P_n} (-1)^{p_1} 
\binom{p_1}{p_2}\binom{p_2}{p_3} \ldots \binom{p_{n-1}}{p_{n}} 
\left(\frac{1}{1 \cdot 2}\right)^{p_1} \left(\frac{1}{3 \cdot 4}\right)^{p_2}  \ldots
\left(\frac{1}{(2n-1)2n}\right)^{p_n} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A000364|id:A241171">A000364, A241171</a>, Euler (or secant) numbers with even index. Note that we have already seen the triangle under the name "Joffe's central differences of zero" in Formula 2.</p> 

In [37]:
nrm = lambda n, k: factorial(2 * n)

for n in range(6): print(PartitionSum(n, eul, nrm, v=x))
print()
for n in range(6): print(PartitionSum(n, eul, nrm, v=1))
print()
print([sum(s) for s in [PartitionSum(n, eul, nrm, v=-1) for n in range(9)]])

1
x
6*x^2 + x
90*x^3 + 30*x^2 + x
2520*x^4 + 1260*x^3 + 126*x^2 + x
113400*x^5 + 75600*x^4 + 13230*x^3 + 510*x^2 + x

[1]
[0, 1]
[0, 1, 6]
[0, 1, 30, 90]
[0, 1, 126, 1260, 2520]
[0, 1, 510, 13230, 75600, 113400]

[1, -1, 5, -61, 1385, -50521, 2702765, -199360981, 19391512145]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>5</span></h2>

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A036280|id:A036281">A036280/A036281</a>, Taylor series for x * cosec(x), coefficients appearing in the MacLaurin summation formula (might be called the 'MacLaurin numbers').</p>

In [38]:
bern = lambda n: 1 / ((2 * n) * (2 * n + 1))

print([PartitionSum(n, bern, v=x) for n in range(4)])
print([PartitionSum(n, bern, v=-1) for n in range(4)])
print([sum(s) for s in [PartitionSum(n, bern, v=-1) for n in range(8)]])

[1, 1/6*x, 1/36*x^2 + 1/120*x, 1/216*x^3 + 1/360*x^2 + 1/5040*x]
[[1], [0, -1/6], [0, -1/120, 1/36], [0, -1/5040, 1/360, -1/216]]
[1, -1/6, 7/360, -31/15120, 127/604800, -73/3421440, 1414477/653837184000, -8191/37362124800]


$$ B_{2n} = \frac{(2n)!}{(2-2^{2n})} \sum_{p \in P_n} (-1)^{p_1} 
\binom{p_1}{p_2}\binom{p_2}{p_3} \ldots \binom{p_{n-1}}{p_{n}} 
\left(\frac{1}{2 \cdot 3}\right)^{p_1} \left(\frac{1}{4 \cdot 5}\right)^{p_2}  
\ldots \left(\frac{1}{2n  (2n+1)}\right)^{p_n} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A000367|id:A002445">A000367/A002445</a>, Bernoulli numbers with even index.</p>

In [39]:
norm = lambda n, k: factorial(2 * n) / (2 - 2^(2 * n))

for n in range(5): print(PartitionSum(n, bern, norm, v=x)) 
print()
for n in range(5): print(PartitionSum(n, bern, norm, v=-1))
print()
print([sum(s) for s in 
        [PartitionSum(n, bern, norm, v=-1) for n in range(8)]])

1
-1/6*x
-1/21*x^2 - 1/70*x
-5/93*x^3 - 1/31*x^2 - 1/434*x
-140/1143*x^4 - 14/127*x^3 - 41/1905*x^2 - 1/2286*x

[1]
[0, 1/6]
[0, 1/70, -1/21]
[0, 1/434, -1/31, 5/93]
[0, 1/2286, -41/1905, 14/127, -140/1143]

[1, 1/6, -1/30, 1/42, -1/30, 5/66, -691/2730, 7/6]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
PART 5 Stirling and Lah numbers of higher order<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/P-Transform#.E2.99.A6.C2.A0_Stirling_and_Lah_numbers_of_higher_order">OEIS-blog</a>]</span></h3>

As prototypical examples for the $P$ transformation we choose the unsigned Lah numbers,
the Stirling set and the (unsigned) Stirling cycle numbers and Kronecker's delta. 
We show these examples in the form taken by the partial $P$-polynomials. 
This time we use a more mathematical notation. So let's define 
$ \mathcal{P}^k_n(f) $ as the $P$ transform of $f$ restricted to the partitions 
of $n$ with largest part $k$; this is exactly what the partial $P$-polynomials generated
by $f$ represent. Then:

Signless Lah numbers: 
$$  \frac{n!}{k!}\, \mathcal{P}^k_n \left(1,1,1,\ldots \right) = \left| {n\atop k} \right| $$

Stirling cycle numbers: 
$$ \frac{n!}{k!}\, \mathcal{P}^k_n \left(1,\frac12,\frac23,\ldots \right) = \left[ {n\atop k} \right] $$

Stirling set numbers: 
$$ \frac{n!}{k!}\, \mathcal{P}^k_n \left(1,\frac12,\frac13,\ldots \right) = \left\{ {n\atop k} \right\} $$

Kronecker delta: 
$$ \frac{n!}{k!}\, \mathcal{P}^k_n (1,0,0, \ldots) = \delta_{n,k} $$

<p>There is a natural way to generalize the unsigned Lah numbers, the Stirling cycle
numbers and the Stirling set numbers to a full hierarchy of combinatorial numbers in 
a uniform way (I described this elsewhere). The case m = 2 in this hierarchy gives 
for the Stirling set numbers the central factorial numbers T(2n, 2k) in Riordan's 
notation (A036969) and for the Stirling cycle numbers the central factorial numbers
t(2n, 2k) (A204579). The case of the Lah numbers has not been considered before as
far as I know (the numbers are now registered as A268434). All these cases
can be computed with the partial P-polynomials as easily as the base cases.</p>

Signless Lah numbers of order 2:

$$ \frac{(2n)!}{(2k)!}\, \mathcal{P}^k_n \left(1,\frac{1}{6},\frac{1}{6},\frac{5}{28},\ldots \right) = \left| {n\atop k} \right| ^{[2]} $$

Stirling cycle numbers of order 2 (a.k.a. central factorial numbers t(2n, 2k)):

$$ \frac{(2n)!}{(2k)!}\, \mathcal{P}^k_n \left(1,\frac{1}{12},\frac{2}{15},\frac{9}{56},\ldots \right) = \left[ {n\atop k} \right] ^{[2]} $$

Stirling set numbers of order 2 (a.k.a. central factorial numbers T(2n, 2k)):

$$ \frac{(2n)!}{(2k)!}\, \mathcal{P}^k_n \left(1,\frac{1}{12},\frac{1}{30},\frac{1}{56},\ldots \right) = \left\{ {n\atop k} \right\} ^{[2]} $$

But why call $ \left| {n\atop k} \right| ^{[2]} $ the *Lah* numbers of order 2?
Well, there are several convincing arguments that this is the 'right' generalization of the Lah numbers. We indicate just the simplest one: Lah numbers of order 1 tie in with the Stirling numbers of order 1 by

$$ \left| {n\atop k} \right| = \sum\limits_{j\,=\,k}^{n} \left[ {n\atop j} \right] \left\{{j\atop k} \right\} \, , $$
and Lah numbers of order 2 have the representation 

$$ \left| {n\atop k} \right| ^{[2]}  =  \sum\limits_{j\,=\,k}^{n} \left[ {n\atop j} \right] ^{[2]}  \left\{ {j\atop k} \right\} ^{[2]}\, . $$

We are inclined to say that a generalization of the Stirling numbers is only satisfying 
if it leads simultaneously to a satisfactory generalization of the Lah numbers.


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>6</span></h2>

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A269945|id:A269944|id:A268434">A269945, A269944, A268434</a></p>

* Stirling set numbers of order 2, 
* Stirling cycle numbers of order 2, 
* Lah numbers of order 2, 

and their inverse. Stirling cycle numbers of order 2 are also known as **central factorial numbers t(2n, 2k)** and Stirling set numbers of order 2 are also known as **central factorial numbers T(2n, 2k)**.

<p>The generators and the normalization:</p>

In [40]:
stirset2   = lambda n: 1 if n == 1 else 1 / (n * (4 * n - 2)) 
stircycle2 = lambda n: 1 if n == 1 else (n - 1)^2 / (n * (4 * n - 2)) 
lah2       = lambda n: 1 if n == 1 else (1 + (n - 1)^2) / (n * (4 * n - 2)) 
norm       = lambda n,k: factorial(2 * n) // factorial(2 * k)

In [41]:
M = PartitionTransform(7, stirset2, norm, accu=True)
for m in M: print(m)

print()

M = PartitionTransform(7, stirset2, norm, accu=True, inverse=True)
for m in M: print(m)

In: [1, 1/12, 1/360, 1/20160, 1/1814400, 1/239500800]
[1]
[0, 1]
[0, 1, 1]
[0, 1, 5, 1]
[0, 1, 21, 14, 1]
[0, 1, 85, 147, 30, 1]
[0, 1, 341, 1408, 627, 55, 1]

In: [1, 1/12, 1/360, 1/20160, 1/1814400, 1/239500800]
[1]
[0, 1]
[0, -1, 1]
[0, 4, -5, 1]
[0, -36, 49, -14, 1]
[0, 576, -820, 273, -30, 1]
[0, -14400, 21076, -7645, 1023, -55, 1]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>7</span></h2>

In [42]:
M = PartitionTransform(7, stircycle2, norm, accu=True)
for m in M: print(m)

print()

M = PartitionTransform(7, stircycle2, norm, accu=True, inverse=True)
for m in M: print(m)

In: [1, 1/12, 1/90, 1/560, 1/3150, 1/16632]
[1]
[0, 1]
[0, 1, 1]
[0, 4, 5, 1]
[0, 36, 49, 14, 1]
[0, 576, 820, 273, 30, 1]
[0, 14400, 21076, 7645, 1023, 55, 1]

In: [1, 1/12, 1/90, 1/560, 1/3150, 1/16632]
[1]
[0, 1]
[0, -1, 1]
[0, 1, -5, 1]
[0, -1, 21, -14, 1]
[0, 1, -85, 147, -30, 1]
[0, -1, 341, -1408, 627, -55, 1]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>8</span></h2>

In [43]:
M = PartitionTransform(7, lah2, norm, accu=True)
for m in M: print(m)

print()

M = PartitionTransform(7, lah2, norm, accu=True, inverse=True)
for m in M: print(m)

In: [1, 1/6, 1/36, 5/1008, 17/18144, 221/1197504]
[1]
[0, 1]
[0, 2, 1]
[0, 10, 10, 1]
[0, 100, 140, 28, 1]
[0, 1700, 2900, 840, 60, 1]
[0, 44200, 85800, 31460, 3300, 110, 1]

In: [1, 1/6, 1/36, 5/1008, 17/18144, 221/1197504]
[1]
[0, 1]
[0, -2, 1]
[0, 10, -10, 1]
[0, -100, 140, -28, 1]
[0, 1700, -2900, 840, -60, 1]
[0, -44200, 85800, -31460, 3300, -110, 1]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;"></h3>

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>9</span></h2>

Let us consider also briefly the third step in the hierarchy,  but this time focusing on the structure of the recurrence relation behind these numbers.

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A269948|id:A269947|id:A269946">A269948, A269947, A269946</a></p> 

* Stirling set numbers of order 3, 
* Stirling cycle numbers of order 3, and 
* Lah numbers of order 3 

and their inverse.

In [44]:
stirset3 = lambda n, k: k^3
stircycle3 = lambda n, k: (n - 1)^3
lah3 = lambda n, k: stircycle3(n, k) + stirset3(n, k)

In [45]:
def T(n, k, w):
    if n == k: return 1
    if k < 0 or k > n: return 0
    return T(n - 1, k - 1, w) + w(n, k) * T(n - 1, k, w)

for n in range(6):
    print([T(n, k, stirset3) for k in range(n + 1)])

for n in range(6):
    print([T(n, k, stircycle3) for k in range(n + 1)])
    
for n in range(6):
    print([T(n, k, lah3) for k in range(n + 1)])

[1]
[0, 1]
[0, 1, 1]
[0, 1, 9, 1]
[0, 1, 73, 36, 1]
[0, 1, 585, 1045, 100, 1]
[1]
[0, 1]
[0, 1, 1]
[0, 8, 9, 1]
[0, 216, 251, 36, 1]
[0, 13824, 16280, 2555, 100, 1]
[1]
[0, 1]
[0, 2, 1]
[0, 18, 18, 1]
[0, 504, 648, 72, 1]
[0, 32760, 47160, 7200, 200, 1]


</p>
<table style="width: 100%; text-align:center; background-color:#CDE1BA ">  
<caption style="caption-side:top; font-style:italic">The hierarchy of the Stirling/Lah numbers.</caption>
<tr>
<td style="background-color:#FFD700; color:blue">Order</td>
<td style="background-color:#FFD700; color:blue">Stirling set </td>
<td style="background-color:#FFD700; color:blue">Stirling cycle</td>
<td style="background-color:#FFD700; color:blue">Lah numbers</td>
</tr>
<tr>
<td style="background-color:#FFD700; color:blue">0</td>
<td>A007318</td>
<td>A007318</td>
<td>A038207</td>
</tr>
<tr>
<td style="background-color:#FFD700; color:blue">1</td>
<td>A048993</td>
<td>A132393</td>
<td>A271703</td>
</tr>
<tr>
<td style="background-color:#FFD700; color:blue">2</td>
<td>A269945</td>
<td>A269944</td>
<td>A268434</td>
</tr>
<tr>
<td style="background-color:#FFD700; color:blue">3</td>
<td>A269948</td>
<td>A269947</td>
<td>A269946</td>
</tr>
</table>

In particular we see that the case m = 0 
reduces both kinds of Stirling numbers to Pascal's triangle 
and that the Lah numbers of order 0 are given by A038207, 
which is the square of Pascal's triangle.

In a compact format all three types for all orders:

In [46]:
@cache
def StirlingLah(typ, m, n, k):
    if n == k: return 1 
    if k < 0 or k > n: return 0 
    if   typ == "St1": r = (n - 1)**m
    elif typ == "St2": r = k**m
    elif typ == "Lah": r = (n - 1)**m + k**m
    return ( StirlingLah(typ, m, n - 1, k - 1)
           + StirlingLah(typ, m, n - 1, k) * r)

oeis = iter([
    "A007318","A048993","A269945","A269948",
    "A007318","A132393","A269944","A269947",
    "A038207","A271703","A268434","A269946"
])

def StirlingLahFamily():
    for typ in ["St1", "St2", "Lah"]:
        for order in range(4):
            print(f"\nTyp:{typ}, Order:{order}, OEIS:{next(oeis)}")
            for n in range(7): 
                print([StirlingLah(typ, order, n, k) for k in range(n + 1)])

# StirlingLahFamily()  # to execute the test remove comment sign

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A271703">A271703</a> For easy referencing later we define the classical (unsigned) Lah numbers explicitly.</p>

In [47]:
def lah_number(n, k): 
    return binomial(n, k) * falling_factorial(n - 1, n - k)

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
The StirlingStar numbers<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/P-Transform#.E2.99.A6.C2.A0_Some_additional_Stirling_number_identities">OEIS-blog</a>]</span></h3>

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>10</span></h2>

*Definitions:*

$$ \left| {n\atop k} \right|^{\star}  =  (2n)! \, \mathcal{P}^k_n \left(1, 1, 1, 1,\ldots \right)$$
$$ \left[ {n\atop k} \right]^{\star} = (2n)! \, \mathcal{P}^k_n \left(1,\frac12,\frac23,\frac34,\ldots \right) $$
$$ \left\{ {n\atop k} \right\}^{\star} = (2n)! \, \mathcal{P}^k_n \left(1,\frac12,\frac13,\frac14,\ldots \right) $$

We call these numbers **LahStar**, **StirlingSetStar**, and **StirlingCycleStar**. These names were invented here only to be able to reference them easily, there may be better names. The numerical values of the StirlingStar numbers can be found in A268437 and A268438. Considering the simplicity of their partition representation, it seems that they have received too little attention so far.

In [48]:
LahStarGenerator           = lambda n: 1
StirlingSetStarGenerator   = lambda n: 1 / (n + 1)
StirlingCycleStarGenerator = lambda n: n / (n + 1)

StarNorm = lambda n, k: factorial(2 * n)

In [49]:
print(PartitionTransform(7, LahStarGenerator, StarNorm, accu = True))
print()
print(PartitionTransform(7, StirlingSetStarGenerator, StarNorm, accu = True))
print()
print(PartitionTransform(7, StirlingCycleStarGenerator, StarNorm, accu = True))

In: [1, 1, 1, 1, 1, 1]
[[1], [0, 2], [0, 24, 24], [0, 720, 1440, 720], [0, 40320, 120960, 120960, 40320], [0, 3628800, 14515200, 21772800, 14515200, 3628800], [0, 479001600, 2395008000, 4790016000, 4790016000, 2395008000, 479001600]]

In: [1/2, 1/6, 1/24, 1/120, 1/720, 1/5040]
[[1], [0, 1], [0, 4, 6], [0, 30, 120, 90], [0, 336, 2800, 5040, 2520], [0, 5040, 80640, 264600, 302400, 113400], [0, 95040, 2827440, 15190560, 29937600, 24948000, 7484400]]

In: [1/2, 1/3, 1/4, 1/5, 1/6, 1/7]
[[1], [0, 1], [0, 8, 6], [0, 180, 240, 90], [0, 8064, 14560, 10080, 2520], [0, 604800, 1330560, 1285200, 604800, 113400], [0, 68428800, 173638080, 209341440, 139708800, 49896000, 7484400]]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">The Circ and the Star Transform</h3>

The 'circ transform' is a triangle to triangle transformation, (as usual, 'triangle' means a lower infinite triangular array T(n, k) of integers, where 0 <= k <= n.) It is defined:

$$ \operatorname{T}^{\circ}(n,k) \, = \, \sum_{m=0}^{k} (-1)^{m+k} 
\binom{n+k}{n+m} \operatorname{T}(n+m, m) $$

The 'star transform' scales the 'circ transform' with the factor 
$ \frac{(2n)!}{(n+k)^{\underline{n}}} $.

$$ \operatorname{T}^{\star}(n,k) \, = \, \frac{(2n)!}{(n+k)^{\underline{n}}} \, \sum_{m=0}^{k} (-1)^{m+k} \binom{n+k}{n+m} \operatorname{T}(n+m, m)$$

In [50]:
def Tcirc(T: Callable[[int, int], int]) -> Callable[[int, int], int]:
    def tcirc(n: int, k: int) -> int:
        return sum(
            (-1)^(m + k) * binomial(n + k, n + m) * T(n + m, m) for m in range(k + 1)
        )
    return tcirc


def Tstar(T: Callable[[int, int], int]) -> Callable[[int, int], int]:
    def tstar(n: int, k: int) -> int:
        return ( sum(
            (-1)^(m + k) * binomial(n + k, n + m) * T(n + m, m) for m in range(k + 1))
            * factorial(2 * n) // falling_factorial(n + k, n) 
        )
    return tstar


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>11</span></h2>

$$ \left| {n\atop k} \right|^{\star}  =  (2n)! \, \mathcal{P}^k_n \left(1, 1, 1, 1,\ldots \right)$$

$$ \left| {n \atop k} \right|^{\star}
= \frac{(2n)!}{(n+k)^{\underline{n}}} \sum_{m=0}^{k} (-1)^{m+k} \binom{n+k}{n+m}
\left| {n +m \atop m} \right| $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/A357367">A357367, missing</a>, Lah circ and Lah star.</p>

In [51]:
lah_circ = Tcirc(lah_number)
lah_star = Tstar(lah_number)

for n in range(7):
    print([lah_circ(n, k) for k in range(n+1)])
    
print()
for n in range(7):
    print([lah_star(n, k) for k in range(n+1)])

print()
PartitionTransform(7, LahStarGenerator, StarNorm, accu = True)

[1]
[0, 2]
[0, 6, 12]
[0, 24, 120, 120]
[0, 120, 1080, 2520, 1680]
[0, 720, 10080, 40320, 60480, 30240]
[0, 5040, 100800, 604800, 1512000, 1663200, 665280]

[1]
[0, 2]
[0, 24, 24]
[0, 720, 1440, 720]
[0, 40320, 120960, 120960, 40320]
[0, 3628800, 14515200, 21772800, 14515200, 3628800]
[0, 479001600, 2395008000, 4790016000, 4790016000, 2395008000, 479001600]

In: [1, 1, 1, 1, 1, 1]


[[1],
 [0, 2],
 [0, 24, 24],
 [0, 720, 1440, 720],
 [0, 40320, 120960, 120960, 40320],
 [0, 3628800, 14515200, 21772800, 14515200, 3628800],
 [0, 479001600, 2395008000, 4790016000, 4790016000, 2395008000, 479001600]]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>12</span></h2>

$$ \left[ {n\atop k} \right]^{\star} = (2n)! \, \mathcal{P}^k_n \left(1,\frac12,\frac23,\frac34,\ldots \right) $$

$$ \genfrac{ [ }{ ] }{0pt}{}{n}{k}^{\star} 
= \frac{(2n)!}{(n+k)^{\underline{n}}} \sum_{m=0}^{k} (-1)^{m+k} \binom{n+k}{n+m}\genfrac{[ }{ ] }{0pt}{}{n+m}{m} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A269940|id:A268438">A269940, A268438</a>, 
Stirling cycle circ and Stirling cycle star.</p>

In [52]:
st1_circ = Tcirc(stirling_number1)
st1_star = Tstar(stirling_number1)

for n in range(7):
    print([st1_circ(n, k) for k in range(n+1)])

print()
for n in range(7):
    print([st1_star(n, k) for k in range(n+1)])

print()
PartitionTransform(7, StirlingCycleStarGenerator, StarNorm, accu = True)

[1]
[0, 1]
[0, 2, 3]
[0, 6, 20, 15]
[0, 24, 130, 210, 105]
[0, 120, 924, 2380, 2520, 945]
[0, 720, 7308, 26432, 44100, 34650, 10395]

[1]
[0, 1]
[0, 8, 6]
[0, 180, 240, 90]
[0, 8064, 14560, 10080, 2520]
[0, 604800, 1330560, 1285200, 604800, 113400]
[0, 68428800, 173638080, 209341440, 139708800, 49896000, 7484400]

In: [1/2, 1/3, 1/4, 1/5, 1/6, 1/7]


[[1],
 [0, 1],
 [0, 8, 6],
 [0, 180, 240, 90],
 [0, 8064, 14560, 10080, 2520],
 [0, 604800, 1330560, 1285200, 604800, 113400],
 [0, 68428800, 173638080, 209341440, 139708800, 49896000, 7484400]]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>13</span></h2>

$$ \left\{ {n\atop k} \right\}^{\star} = (2n)! \, \mathcal{P}^k_n \left(1,\frac12,\frac13,\frac14,\ldots \right) $$

$$ \genfrac\{\}{0pt}{}{n}{k}^{\star}
= \frac{(2n)!}{(n+k)^{\underline{n}}} \sum_{m=0}^{k} (-1)^{m+k} \binom{n+k}{n+m}\genfrac\{\}{0pt}{}{n+m}{m} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS</span>
<a href="https://oeis.org/search?q=id:A269939|id:A268437">A269939, A268437</a>, Ward numbers and Stirling set star.</p>

In [53]:
st2_circ = Tcirc(stirling_number2)
st2_star = Tstar(stirling_number2)

for n in range(7):
    print([st2_circ(n, k) for k in range(n+1)])

print()
for n in range(7):
    print([st2_star(n, k) for k in range(n+1)])

print()
PartitionTransform(7, StirlingSetStarGenerator, StarNorm, accu = True)

[1]
[0, 1]
[0, 1, 3]
[0, 1, 10, 15]
[0, 1, 25, 105, 105]
[0, 1, 56, 490, 1260, 945]
[0, 1, 119, 1918, 9450, 17325, 10395]

[1]
[0, 1]
[0, 4, 6]
[0, 30, 120, 90]
[0, 336, 2800, 5040, 2520]
[0, 5040, 80640, 264600, 302400, 113400]
[0, 95040, 2827440, 15190560, 29937600, 24948000, 7484400]

In: [1/2, 1/6, 1/24, 1/120, 1/720, 1/5040]


[[1],
 [0, 1],
 [0, 4, 6],
 [0, 30, 120, 90],
 [0, 336, 2800, 5040, 2520],
 [0, 5040, 80640, 264600, 302400, 113400],
 [0, 95040, 2827440, 15190560, 29937600, 24948000, 7484400]]

<center><table style="text-align:center">
    <caption><b>Summary</b></caption>
    <tr style="font-weight:650">
        <td>Typ</td>
        <td>Lah</td>
        <td>StirCycle</td>
        <td>StirSet</td>
    </tr>
    <tr>
        <td><b>def</b></td>
        <td><a href="https://oeis.org/A271703">A271703</a></td>
        <td><a href="https://oeis.org/A132393">A132393</a></td>
        <td><a href="https://oeis.org/A048993">A048993</a></td>
    </tr>
    <tr>
        <td><b>circ</b></td>
        <td><a href="https://oeis.org/A357367">A357367</a></td>
        <td><a href="https://oeis.org/A269940">A269940</a></td>
        <td><a href="https://oeis.org/A269939">A269939</a></td>
    </tr>
    <tr>
        <td><b>star</b></td>
        <td><a href="https://oeis.org"> --- </a></td>
        <td><a href="https://oeis.org/A268438">A268438</a></td>
        <td><a href="https://oeis.org/A268437">A268437</a></td>
    </tr>
</table></center>

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Extension of the Binomial<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/ExtensionsOfTheBinomial">OEIS-blog</a>]</span></h3>

In [54]:
def b(n, k): return factorial(n) // (factorial(k) * factorial(n-k))

def Binomial(n, k):
    if 0 <= k <= n: return b(n, k)                    # Pascal
    if k <= n <  0: return b(-k - 1, n - k)           # Riordan
    if n <  0 <= k: return b(-n + k - 1, k) * (-1)^k  # Taylor
    return 0

for n in range(-5, 5):
    print([Binomial(n, k) for k in range(-5, 5)])

[1, 0, 0, 0, 0, 1, -5, 15, -35, 70]
[4, 1, 0, 0, 0, 1, -4, 10, -20, 35]
[6, 3, 1, 0, 0, 1, -3, 6, -10, 15]
[4, 3, 2, 1, 0, 1, -2, 3, -4, 5]
[1, 1, 1, 1, 1, 1, -1, 1, -1, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 2, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 3, 3, 1, 0]
[0, 0, 0, 0, 0, 1, 4, 6, 4, 1]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;"></h3>

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>14</span></h2>

$$(n-k+1)^{\overline{n-k}} \, \genfrac\{\}{0pt}{}{n}{k} 
= \binom{n}{k} \sum_{i=0}^{k} \binom{k}{i} \genfrac\{\}{0pt}{}{n-k}{i}^{\star} 
= \binom{-k}{-n} \sum_{i=0}^{n-k} (-1)^{n - k} \binom{-n}{i} \genfrac{[ }{ ] }{0pt}{}{n-k}{i}^{\star} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS </span><a href="https://oeis.org/search?q=id:A268435|id:A357340">A268435, A357340</a>.</p>

*First representation:*

In [55]:
def T(n, k): 
    return rising_factorial(n - k + 1, n - k) * stirling_number2(n, k)

for n in range(8): print([T(n, k) for k in range(n + 1)])

[1]
[0, 1]
[0, 2, 1]
[0, 12, 6, 1]
[0, 120, 84, 12, 1]
[0, 1680, 1800, 300, 20, 1]
[0, 30240, 52080, 10800, 780, 30, 1]
[0, 665280, 1905120, 505680, 42000, 1680, 42, 1]


*Second representation:*

In [56]:
def T(n, k):
    return (binomial(n, k) * sum(binomial(k, i) * st2_star(n - k, i) 
            for i in range(k + 1)))

for n in range(8): print([T(n, k) for k in range(n + 1)])

[1]
[0, 1]
[0, 2, 1]
[0, 12, 6, 1]
[0, 120, 84, 12, 1]
[0, 1680, 1800, 300, 20, 1]
[0, 30240, 52080, 10800, 780, 30, 1]
[0, 665280, 1905120, 505680, 42000, 1680, 42, 1]


*Third representation:*

In [57]:
def A357340(n, k):
    return sum((-1)^(n - k) * binomial(-n, i) * st1_star(n - k, i) 
           for i in  range(n - k + 1))

def T(n, k): return Binomial(-k, -n) * A357340(n, k)

for n in range(7): print([n],[A357340(n, k) for k in range(n + 1)])
for n in range(8): print([n], [T(n, k) for k in range(n + 1)])

[0] [1]
[1] [1, 1]
[2] [2, 2, 1]
[3] [0, 12, 3, 1]
[4] [-56, 120, 28, 4, 1]
[5] [0, 1680, 450, 50, 5, 1]
[6] [15840, 30240, 10416, 1080, 78, 6, 1]
[0] [1]
[1] [0, 1]
[2] [0, 2, 1]
[3] [0, 12, 6, 1]
[4] [0, 120, 84, 12, 1]
[5] [0, 1680, 1800, 300, 20, 1]
[6] [0, 30240, 52080, 10800, 780, 30, 1]
[7] [0, 665280, 1905120, 505680, 42000, 1680, 42, 1]


<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>15</span></h2>

$$ (n-k+1)^{\overline{n-k}} \, \genfrac{ [ }{ ] }{0pt}{}{n}{k} 
= \binom{n}{k} \sum_{i=0}^{k} \binom{k}{i} \genfrac{ [ }{ ] }{0pt}{}{n-k}{i}^{\star}
= \binom{-k}{-n} \sum_{i=0}^{n-k} (-1)^{n - k} \binom{-n}{i} \genfrac\{\}{0pt}{}{n-k}{i}^{\star} $$

<p><span style='color:#5E7AFF; font-weight:bold'>OEIS </span><a href="https://oeis.org/search?q=id:A357339|id:A268436">A268436, A357339</a>.</p>

*First representation:*

In [58]:
def T(n, k): 
    return rising_factorial(n - k + 1, n - k) * stirling_number1(n, k)

for n in range(7): print([T(n, k) for k in range(n + 1)])

[1]
[0, 1]
[0, 2, 1]
[0, 24, 6, 1]
[0, 720, 132, 12, 1]
[0, 40320, 6000, 420, 20, 1]
[0, 3628800, 460320, 27000, 1020, 30, 1]


*Second representation:*

In [59]:
def T(n, k):
    return (binomial(n, k) * sum(binomial(k, i) * st1_star(n - k, i) 
        for i in range(k + 1)))

for n in range(7): print([T(n, k) for k in range(n + 1)])

[1]
[0, 1]
[0, 2, 1]
[0, 24, 6, 1]
[0, 720, 132, 12, 1]
[0, 40320, 6000, 420, 20, 1]
[0, 3628800, 460320, 27000, 1020, 30, 1]


<p><em>Third representation: </em></p>

In [60]:
def A357339(n, k):
    return sum((-1)^(n - k) * binomial(-n, i) * st2_star(n - k, i) 
        for i in range(n - k + 1))

def T(n, k): return Binomial(-k, -n) * A357339(n, k)        

for n in range(7): print([n], [A357339(n, k) for k in range(n + 1)])
for n in range(7): print([n], [T(n, k) for k in range(n + 1)])

[0] [1]
[1] [1, 1]
[2] [10, 2, 1]
[3] [270, 24, 3, 1]
[4] [14056, 720, 44, 4, 1]
[5] [1197000, 40320, 1500, 70, 5, 1]
[6] [151169040, 3628800, 92064, 2700, 102, 6, 1]
[0] [1]
[1] [0, 1]
[2] [0, 2, 1]
[3] [0, 24, 6, 1]
[4] [0, 720, 132, 12, 1]
[5] [0, 40320, 6000, 420, 20, 1]
[6] [0, 3628800, 460320, 27000, 1020, 30, 1]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Appendix:<br>Some partition polynomials evaluated at x=1 or x=-1<span style="font-size:small">
[see <a href="https://oeis.org/wiki/User:Peter_Luschny/P-Transform#.E2.99.A6.C2.A0Some_P-polynomials_evaluated_at_x.3D1_or_x.3D-1">OEIS-blog</a>]</span></h3>

<table style="width: 100%; text-align:center; background-color:#CDE1BA ">  <tr>
<td><a href="https://oeis.org/A000007">A000007</a></td>
<td><a href="https://oeis.org/A000670">A000670</a></td>
<td><a href="https://oeis.org/A001896">A001896</a></td>
<td><a href="https://oeis.org/A003319">A003319</a></td>
<td><a href="https://oeis.org/A006153">A006153</a></td>
<td><a href="https://oeis.org/A006232">A006232</a></td></tr><tr>
<td><a href="https://oeis.org/A006252">A006252</a></td>
<td><a href="https://oeis.org/A006568">A006568</a></td>
<td><a href="https://oeis.org/A006569">A006569</a></td>
<td><a href="https://oeis.org/A007840">A007840</a></td>
<td><a href="https://oeis.org/A027641">A027641</a></td>
<td><a href="https://oeis.org/A027642">A027642</a></td></tr><tr>
<td><a href="https://oeis.org/A036280">A036280</a></td>
<td><a href="https://oeis.org/A036281">A036281</a></td>
<td><a href="https://oeis.org/A036968">A036968</a></td>
<td><a href="https://oeis.org/A051296">A051296</a></td>
<td><a href="https://oeis.org/A075178">A075178</a></td>
<td><a href="https://oeis.org/A077607">A077607</a></td></tr><tr>
<td><a href="https://oeis.org/A089148">A089148</a></td>
<td><a href="https://oeis.org/A101686">A101686</a></td>
<td><a href="https://oeis.org/A113871">A113871</a></td>
<td><a href="https://oeis.org/A118196">A118196</a></td>
<td><a href="https://oeis.org/A135920">A135920</a></td>
<td><a href="https://oeis.org/A154288">A154288</a></td></tr><tr>
<td><a href="https://oeis.org/A154289">A154289</a></td>
<td><a href="https://oeis.org/A167894">A167894</a></td>
<td><a href="https://oeis.org/A198631">A198631</a></td>
<td><a href="https://oeis.org/A226158">A226158</a></td>
<td><a href="https://oeis.org/A248964">A248964</a></td>
<td><a href="https://oeis.org/A249024">A249024</a></td></tr></table>

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">Appendix: Interactive utilities<span style="font-size:small"></span></h3>

You can query the OEIS (Online Database of Integer Sequences) through the two functions below. Internet access is a prerequisite. 

The 'PartitionExplorer' extends the 'EvalAll' function to any valid OEIS A-number. 

The 'PartitionResearcher' is an elaborated version of the 'PartitionExplorer' which goes one level deeper trying to identify also the generated sequences.

In [65]:
from sage.databases.oeis import oeis
oeis(53)  # What?

A000053: Local stops on New York City Broadway line (IRT #1) subway.

In [62]:
def PartitionExplorer(anum: str):
    seq = oeis(anum)
    a = seq.first_terms(10)
    EvalAll(8, lambda n: a[n])


# Please enter a OEIS A-number:
PartitionExplorer('A000055')


PARTPOLY
[[1], [0, 1], [0, 1, 1], [0, 1, 2, 1], [0, 2, 3, 3, 1], [0, 3, 6, 6, 4, 1], [0, 6, 11, 13, 10, 5, 1], [0, 11, 22, 27, 24, 15, 6, 1]]

MONOLIST
[[1], [0, 1], [0, 1, 1], [0, 1, 2, 1], [0, 2, 2, 1, 3, 1], [0, 3, 4, 2, 3, 3, 4, 1], [0, 6, 6, 4, 1, 6, 6, 1, 4, 6, 5, 1], [0, 11, 12, 6, 4, 9, 12, 3, 3, 8, 12, 4, 5, 10, 6, 1]]

MATRIX
[ 1  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0]
[ 0  1  1  0  0  0  0  0]
[ 0  1  2  1  0  0  0  0]
[ 0  2  3  3  1  0  0  0]
[ 0  3  6  6  4  1  0  0]
[ 0  6 11 13 10  5  1  0]
[ 0 11 22 27 24 15  6  1]

INVMATRIX
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0  -1   1   0   0   0   0   0]
[  0   1  -2   1   0   0   0   0]
[  0  -2   3  -3   1   0   0   0]
[  0   5  -6   6  -4   1   0   0]
[  0 -13  15 -13  10  -5   1   0]
[  0  35 -40  33 -24  15  -6   1]

COMPPOLY
[1, 1, 2, 4, 9, 20, 46, 106]

GENSEQ
[1, 1, 1, 2, 3, 6, 11]

INVSEQ
[1, -1, 1, -2, 5, -13, 35]


In [63]:
def PartitionResearcher(anum: str):
    seq = oeis(anum)
    A = seq.first_terms(10)
    f = lambda n: A[n]
    print("SEQ", anum); print(A)

    P = EvalPartPoly(8, f, rep=PP_Repr.MATRIX)
    row = list(P[6][1:7])
    print(P); print('\n', oeis(row, max_results=4))
    row.reverse(); print('\n', oeis(row, max_results=4))

    IP = EvalPartPoly(8, f, rep=PP_Repr.INVMATRIX)
    row = list(IP[6][1:7])
    print(IP); print('\n', oeis(row, max_results=4))
    row.reverse(); print('\n', oeis(row, max_results=4))

    S = EvalPartPoly(8, f, rep=PP_Repr.COMPPOLY); print(S)
    print('\n', oeis(S[1:], max_results=4)) 
    
    IS = EvalPartPoly(8, f, rep=PP_Repr.INVSEQ); print(IS)
    print('\n', oeis(IS[1:], max_results=4)) 
    print()


# Please enter a OEIS A-number:
PartitionResearcher('A000108')

SEQ A000108
(1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862)

MATRIX
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0   2   1   0   0   0   0   0]
[  0   5   4   1   0   0   0   0]
[  0  14  14   6   1   0   0   0]
[  0  42  48  27   8   1   0   0]
[  0 132 165 110  44  10   1   0]
[  0 429 572 429 208  65  12   1]

 0: A039598: Triangle formed from odd-numbered columns of triangle of expansions of powers of x in terms of Chebyshev polynomials U_n(x). Sometimes called Catalan's triangle.
1: A008313: Triangle of expansions of powers of x in terms of Chebyshev polynomials U_n(x).
2: A128899: Riordan array (1,(1-2x-sqrt(1-4x))/(2x)).
3: A050176: T(n,k) = M0(n+1,k,f(n,k)), where M0(p,q,r) is the number of upright paths from (0,0) to (1,0) to (p,p-q) that meet the line y = x-r and do not rise above it and f(n,k) is the least t such that M0(n+1,k,f) is not 0.

 0: A008315: Catalan triangle read by rows. Also triangle of expansions of powers of x in terms of Chebyshev polyno

In [64]:
PartitionResearcher('A000045')

SEQ A000045
(0, 1, 1, 2, 3, 5, 8, 13, 21, 34)

MATRIX
[ 1  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0]
[ 0  1  1  0  0  0  0  0]
[ 0  2  2  1  0  0  0  0]
[ 0  3  5  3  1  0  0  0]
[ 0  5 10  9  4  1  0  0]
[ 0  8 20 22 14  5  1  0]
[ 0 13 38 51 40 20  6  1]

 0: A037027: Skew Fibonacci-Pascal triangle read by rows.
1: A155161: A Fibonacci convolution triangle: Riordan array (1, x/(1 - x - x^2)). Triangle T(n,k), 0 <= k <= n, read by rows.

 0: A038137: Reflection of A037027: T(n,m) = U(n,n-m), m=0..n, where U is as in A037027.
1: A208336: Triangle of coefficients of polynomials u(n,x) jointly generated with A208337; see the Formula section.

INVMATRIX
[  1   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0]
[  0  -1   1   0   0   0   0   0]
[  0   0  -2   1   0   0   0   0]
[  0   2   1  -3   1   0   0   0]
[  0  -3   4   3  -4   1   0   0]
[  0  -1 -10   5   6  -5   1   0]
[  0  11   4 -21   4  10  -6   1]

 0: A202327: Triangle read by rows, T(n, k) is the coefficient 

<p style="text-align:center;color:blue;font-size:larger;">Happy exploring!</p>